In [2]:
# Set display options to show the full output without truncation
import pandas as pd
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full column width
pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame from being split across lines

In [ ]:
def format_brl(value: float) -> str:
    """
    Converts a float number into Brazilian Real (BRL) format.
    
    Args:
        value (float): The float number to be converted.
        
    Returns:
        str: The formatted string in BRL (e.g., 'R$ 14.547,45').
    """
    return f"R$ {value:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")



def numero_ordinal(n):
    unidades = ["", "primeiro", "segundo", "terceiro", "quarto", "quinto", "sexto", "sétimo", "oitavo", "nono"]
    dezenas = ["", "décimo", "vigésimo", "trigésimo", "quadragésimo", "quinquagésimo", "sexagésimo", "septuagésimo", "octogésimo", "nonagésimo"]
    centenas = ["", "centésimo", "ducentésimo", "trecentésimo", "quadringentésimo", "quingentésimo", "sexcentésimo", "septingentésimo", "octingentésimo", "nongentésimo"]
    milhares = ["milésimo"]  # Para milhares, mantemos apenas o sufixo

    def ordinal_de_unidade(n):
        return unidades[n]

    def ordinal_de_dezena(n):
        dezena = n // 10
        unidade = n % 10
        if unidade == 0:
            return dezenas[dezena]
        else:
            return dezenas[dezena] + " " + unidades[unidade]

    def ordinal_de_centena(n):
        centena = n // 100
        resto = n % 100
        if resto == 0:
            return centenas[centena]
        else:
            if resto < 10:
                return centenas[centena] + " " + ordinal_de_unidade(resto)
            else:
                return centenas[centena] + " " + ordinal_de_dezena(resto)

    def ordinal_de_milhar(n):
        milhar = n // 1000
        resto = n % 1000
        if resto == 0:
            if milhar == 1:
                return "milésimo"
            else:
                return ordinal_de_unidade(milhar) + " " + "milésimo"
        else:
            if resto < 100:
                return ordinal_de_unidade(milhar) + " milésimo " + ordinal_de_centena(resto)
            else:
                return ordinal_de_unidade(milhar) + " milésimo " + ordinal_de_milhar(resto)

    if n < 10:
        return ordinal_de_unidade(n)
    elif n < 100:
        return ordinal_de_dezena(n)
    elif n < 1000:
        return ordinal_de_centena(n)
    elif n < 10000:
        return ordinal_de_milhar(n)
    else:
        return "Número muito grande para ser representado automaticamente"

def Numero_ordinal(n):
    return numero_ordinal(n)[0].upper() + numero_ordinal(n)[1:] 
print(Numero_ordinal(3))

def clean_currency(value):
    try:
        return float(value.replace('R$', '').replace('.', '').replace(',', '.').strip())
    except Exception as e:
        return 0

# # Exemplo de uso:
# print(numero_ordinal(1))     # Saída: primeiro
# print(numero_ordinal(21))    # Saída: vigésimo primeiro
# print(numero_ordinal(105))   # Saída: centésimo quinto
# print(numero_ordinal(999))   # Saída: nongentésimo nonagésimo nono
# print(numero_ordinal(1001))  # Saída: milésimo primeiro
# print(numero_ordinal(2021))  # Saída: segundo milésimo vigésimo primeiro


In [ ]:
df = pd.read_csv('SALARIOS_SERVIDORES_UNB_AGOSTO.csv')
MAT = df[df['DEPARTAMENTO']=='DEPTO MATEMATICA'].copy()
    
def limpar_rem(datafr):
    df = datafr
    df['REMUNERACAO BRUTA'] = df['REMUNERACAO BRUTA'].apply(clean_currency)
    df['REMUNERACAO LIQUIDA'] = df['REMUNERACAO LIQUIDA'].apply(clean_currency)
limpar_rem(MAT)
MAT.drop_duplicates(inplace=True)
MAT_sorted = MAT.sort_values(by='REMUNERACAO BRUTA', ascending=False).reset_index(drop=True)
# # MAT_sorted.head()
# # MAT_sorted.to_csv('MAT_RANKING.csv', index = False)
MAT_sorted['Ranking'] = MAT_sorted['REMUNERACAO BRUTA'].rank(ascending=False, method='first').astype(int)
ranking_list = list(MAT_sorted[['NOME', 'REMUNERACAO BRUTA', 'REMUNERACAO LIQUIDA', 'Ranking']].itertuples(index=False, name=None))
MAT_sorted.describe()
# ranking_list[0]
# with open("ranking_mat.txt", "w") as f:
#     for professor in ranking_list:
#         f.write("{num_lugar} ({lugar}) lugar: {nome}, remuneração bruta de {rem}, remuneração líquida de {rem_liq} \n \n".format(num_lugar = professor[3], nome = professor[0], lugar=Numero_ordinal(professor[3]), rem=format_brl(professor[1]), rem_liq = format_brl(professor[2])))